# Basic usage
This notebook demonstrates basic usage of the mwreverts library.  

## Detecting reverts in a sequence of page edits
At the most basic, the library can detect reverts in a stream of "revision" events.  This is useful for finding out what reverts have taken place in a sequence of edits to a page.  In this case, we'll look at reverts in https://en.wikipedia.org/wiki/Wikipedia:Teahouse using the `detect` method.  This pattern is generally useful if you'd like to look for reverting activity for an entire page.

In [1]:
import mwapi, mwreverts
from itertools import islice  # For limiting the result set

In [2]:
# Gather a page's revisions from the API
session = mwapi.Session("https://en.wikipedia.org", user_agent="mwreverts basic usage script")
response_docs = session.get(action="query",
                            prop="revisions",
                            titles={"Wikipedia:Teahouse"},
                            rvprop={'ids', 'sha1', 'timestamp'},
                            rvdir="newer",  # Direction "newer" is important for detection
                            continuation=True)

# Creates a revsion event iterator of (checksum, rev_doc) pairs
rev_events = ((rev_doc['sha1'], rev_doc) 
              for r_doc in response_docs 
              for page_doc in r_doc['query']['pages'].values()
              for rev_doc in page_doc['revisions']
              if 'sha1' in rev_doc)

# Detect and print reverts
for revert in islice(mwreverts.detect(rev_events), 5):
    print("{0} reverts {1} back to {2}"
          .format(revert.reverting['revid'],
                  [rev_doc['revid'] for rev_doc in revert.reverteds],
                  revert.reverted_to['revid']))

477059985 reverts [477059772] back to 477059735
477108430 reverts [477095691, 477093506, 477090831, 477072862, 477067674, 477063205, 477059985, 477059772, 477059735, 477055557] back to 472895667
477108510 reverts [477108430] back to 477095691
478687134 reverts [478685482] back to 478665756
479660766 reverts [479660552, 479658598] back to 479648108


## Checking the "revert status" of a single edit
An edit can *reverting* other edits, it can be *reverted*, or it can be *reverted_to* by another edit.  The `mwreverts.api.check` method will use a `mwapi.Session` to check on the status of an edit by its `"revid"`.  This pattern is generally useful if you'd like to check on the status of a set of edits that span multiple pages.  For example, if you were looking at the activities of a particular user.

In [3]:
import mwapi, mwreverts.api

In [4]:
# Gather a page's revisions from the API
session = mwapi.Session("https://en.wikipedia.org", user_agent="mwreverts basic usage script")

def print_revert_status(rev_id, reverting, reverted, reverted_to):
    """Prints a nice, pretty version of a revert status."""
    print(str(rev_id) + ":")
    if reverting is not None:
        print(" - reverting {0} other edits".format(len(reverting.reverteds)))
    if reverted is not None:
        print(" - reverted in {revid} by {user}".format(**reverted.reverting))
    if reverted_to is not None:
        print(" - reverted_to in {revid} by {user}".format(**reverted_to.reverting))

# Checking the status of 3 interesting edits seen above
reverting, reverted, reverted_to = mwreverts.api.check(session, 477059772, rvprop={'user'})
print_revert_status(477059772, reverting, reverted, reverted_to)
reverting, reverted, reverted_to = mwreverts.api.check(session, 477059985, rvprop={'user'})
print_revert_status(477059985, reverting, reverted, reverted_to)
reverting, reverted, reverted_to = mwreverts.api.check(session, 477059735, rvprop={'user'})
print_revert_status(477059735, reverting, reverted, reverted_to)

477059772:
 - reverted in 477059985 by Heatherawalls
477059985:
 - reverting 1 other edits
 - reverted in 477108430 by AbigailAbernathy
477059735:
 - reverted in 477108430 by AbigailAbernathy
 - reverted_to in 477059985 by Heatherawalls
